
**ENHANCING SENTIMENT ANALYSIS FOR APP RECOMMENDATION ENHANCEMENTS**


**DATASET USED  : GOOGLE PLAY REVIEWS**

**NAME:ANGEL SUSAN VINO**

**GUIDE:DR.ABHIRAMI S**

**Importing Necessary Libraries**









In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

**Data Loading**

In [3]:
df = pd.read_csv("flipkart_reviews_dataset.csv")
df

,product_id,product_title,rating,summary,review,location,date,upvotes,downvotes
0,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Terrific purchase,1-more flexible2-bass is very high3-sound clar...,Shirala,8 months ago,1390,276
1,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Terrific purchase,Super sound and good looking I like that prize,Visakhapatnam,8 months ago,643,133
2,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Super!,Very much satisfied with the device at this pr...,Kozhikode,"Feb, 2020",1449,328
3,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Super!,"Nice headphone, bass was very good and sound i...",Jaora,7 months ago,160,28
4,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Terrific purchase,Sound quality super battery backup super quali...,New Delhi,8 months ago,533,114
...,...,...,...,...,...,...,...,...,...
9369,ACCEVQZABYWJHRHF,BoAt BassHeads 100 Wired Headset,4,Good quality product,this head phnes give good base in pluged ears ...,Bhimavaram,"Apr, 2018",0,0
9370,ACCEVQZABYWJHRHF,BoAt BassHeads 100 Wired Headset,5,Awesome,This so far is one of the best headphones I've...,Ettimadai,"Apr, 2018",0,0
9371,ACCEVQZABYWJHRHF,BoAt BassHeads 100 Wired Headset,4,Really Nice,I am a BASS LOVER and all others like me read ...,Guntur District,"Apr, 2018",0,0
9372,ACCEVQZABYWJHRHF,BoAt BassHeads 100 Wired Headset,5,Super!,After use of one month........ It is an excell...,Kothamangalam,"Apr, 2018",0,0


**Understanding The Data**

In [4]:
df.shape

(9374, 9)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9374 entries, 0 to 9373
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_id     9374 non-null   object
 1   product_title  9374 non-null   object
 2   rating         9374 non-null   int64 
 3   summary        9374 non-null   object
 4   review         9374 non-null   object
 5   location       8081 non-null   object
 6   date           9374 non-null   object
 7   upvotes        9374 non-null   int64 
 8   downvotes      9374 non-null   int64 
dtypes: int64(3), object(6)
memory usage: 659.2+ KB


In [6]:
df.describe()

,rating,upvotes,downvotes
count,9374.000000,9374.000000,9374.000000
mean,4.094623,8.102731,1.911457
std,1.345756,102.296230,15.389562
min,1.000000,0.000000,0.000000
25%,4.000000,0.000000,0.000000
50%,5.000000,0.000000,0.000000
75%,5.000000,1.000000,0.000000
max,5.000000,7611.000000,435.000000


In [7]:
df.dtypes

,0
product_id,object
product_title,object
rating,int64
summary,object
review,object
location,object
date,object
upvotes,int64
downvotes,int64


In [8]:
df.isnull().sum()

,0
product_id,0
product_title,0
rating,0
summary,0
review,0
location,1293
date,0
upvotes,0
downvotes,0


In [10]:
df['rating'].value_counts()

,count
rating,
5,5395
4,1959
1,1090
3,622
2,308


**Data Preprocessing**

In [ ]:
# Handling missing values
df['reviewCreatedVersion'].fillna('unknown_version', inplace=True)
df['replyContent'].fillna('No Reply', inplace=True)
df['repliedAt'].fillna('No Reply', inplace=True)

# Verify that there are no more missing values
print("\nMissing values after handling:\n", df.isnull().sum())

**Data Cleaning**

In [11]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer

# Download necessary resources from nltk
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize stopwords, lemmatizer, and stemmer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()  # Or you can use SnowballStemmer('english')

# Function to remove emojis
def remove_emojis(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F700-\U0001F77F"  # alchemical symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                           u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                           u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                           u"\U0001FA00-\U0001FA6F"  # Chess symbols
                           u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                           u"\U00002702-\U000027B0"  # Dingbats
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# Define a cleaning function with stemming
def clean_text(text):
    # Remove emojis
    text = remove_emojis(text)

    # Lowercasing
    text = text.lower()

    # Remove special characters, numbers, and punctuation
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenization (split text into words)
    words = text.split()

    # Remove stopwords, apply lemmatization, and then stemming
    words = [stemmer.stem(lemmatizer.lemmatize(word)) for word in words if word not in stop_words]

    # Join words back into a single string
    clean_text = ' '.join(words)

    return clean_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [12]:
#Handling Empty or Short Reviews

In [13]:
# Apply the cleaning function to the 'content' column
df['cleaned_content'] = df['review'].apply(clean_text)

# View the cleaned text
print(df[['review', 'cleaned_content']].head())

                                              review  \
0  1-more flexible2-bass is very high3-sound clar...   
1     Super sound and good looking I like that prize   
2  Very much satisfied with the device at this pr...   
3  Nice headphone, bass was very good and sound i...   
4  Sound quality super battery backup super quali...   

                                     cleaned_content  
0  flexiblebass highsound clariti good batteri ba...  
1                   super sound good look like prize  
2  much satisfi devic price point awesom brand de...  
3  nice headphon bass good sound clear purchas ca...  
4  sound qualiti super batteri backup super quali...  


In [14]:
# Remove rows where 'cleaned_content' is empty
df = df[df['cleaned_content'].str.strip().astype(bool)]

# Optionally, remove reviews that are too short
df = df[df['cleaned_content'].str.split().apply(len) > 2]  # Keeping reviews with more than 2 words

In [16]:
# Define a function to categorize recommendations
def recommend(score):
    if score >= 3:
        return 1  # Recommended for scores 3 and above
    else:
        return 0  # Not Recommended for scores below 3

# Apply the function to create the new 'recommended' column
df['recommended'] = df['rating'].apply(recommend)

# Verify the new column
print(df[['rating', 'recommended']].head())

   rating  recommended
0       5            1
1       5            1
2       5            1
3       5            1
4       5            1


In [17]:
# Check unique values in the 'recommended' column
unique_recommendations = df['recommended'].unique()

# Print the unique values
print("Unique values in 'recommended' column:", unique_recommendations)

Unique values in 'recommended' column: [1 0]


In [18]:
# Count occurrences of each unique value in the 'recommended' column
recommendation_counts = df['recommended'].value_counts()

# Print the counts
print("Counts of unique values in 'recommended' column:\n", recommendation_counts)

Counts of unique values in 'recommended' column:
 recommended
1    6021
0    1173
Name: count, dtype: int64


**Aspect Extraction**

In [19]:
!pip install spacy

In [20]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 25.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [23]:
import spacy
import re
from collections import Counter

# Load spaCy's English model
nlp = spacy.load("en_core_web_sm")

# Function to extract noun phrases (aspects)
def extract_aspects(text):
    doc = nlp(text)
    # Extract noun phrases, which often indicate aspects
    aspects = [chunk.text for chunk in doc.noun_chunks]
    return aspects

# Apply the aspect extraction function
df['aspects'] = df['cleaned_content'].apply(extract_aspects)

# View extracted aspects
df[['review', 'aspects']].head()


,review,aspects
0,1-more flexible2-bass is very high3-sound clar...,"[flexiblebass highsound, good batteri back hou..."
1,Super sound and good looking I like that prize,"[super sound good look, prize]"
2,Very much satisfied with the device at this pr...,[much satisfi devic price point awesom brand d...
3,"Nice headphone, bass was very good and sound i...","[nice headphon bass good sound, purchas call p..."
4,Sound quality super battery backup super quali...,"[super price rang super, money]"


In [24]:
# Flatten the list of all aspects
all_aspects = [aspect for sublist in df['aspects'] for aspect in sublist]

# Count the most common aspects
aspect_counts = Counter(all_aspects)
common_aspects = aspect_counts.most_common(10)  # Top 10 most common aspects

# Display the most common aspects
print(common_aspects)

[('i', 287), ('product', 245), ('sound', 213), ('qualiti', 200), ('u', 173), ('money', 161), ('good product', 110), ('awesom', 104), ('earphon', 99), ('nice product', 83)]


**Train Test Split**

In [25]:
# Splitting into X (features) and y (labels)
X = df['cleaned_content']
y = df['recommended']  # Use the binary recommendation as the label

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.pipeline import Pipeline

In [27]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Vectorization**

In [28]:
# Placeholder list to store results
vectorizer_results = []

# Vectorizers
count_vectorizer = CountVectorizer()
tfidf_vectorizer = TfidfVectorizer()

# NMF requires a pipeline with Tfidf
nmf_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('nmf', NMF(n_components=50, random_state=42))
])

# List of vectorizers to evaluate
vectorizers = {
    "CountVectorizer": count_vectorizer,
    "TF-IDF": tfidf_vectorizer,
    "NMF": nmf_pipeline
}

**Model Training**

In [29]:
pip install xgboost

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [31]:
# Define the algorithms
algorithms = {
    "Logistic Regression": LogisticRegression(),
    "Naive Bayes": MultinomialNB(),
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC(kernel='linear'),  # You can also experiment with 'rbf' kernel
    "AdaBoost": AdaBoostClassifier(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss')  # Avoid the warning for the latest versions
}

# Placeholder to store the results
results = []

# Loop through each vectorizer and each algorithm to build and evaluate the model
for vec_name, vectorizer in vectorizers.items():
    for algo_name, algo in algorithms.items():
        # Create a pipeline with vectorizer and algorithm
        pipeline = Pipeline([
            ('vectorizer', vectorizer),
            ('classifier', algo)
        ])

        # Train the model
        pipeline.fit(X_train, y_train)

        # Make predictions
        y_pred = pipeline.predict(X_test)

        # Calculate accuracy
        accuracy = accuracy_score(y_test, y_pred)

        # Get the confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)

        # Get the classification report (precision, recall, F1-score)
        class_report = classification_report(y_test, y_pred, output_dict=True)

        # Store the results
        results.append({
            'Vectorizer': vec_name,
            'Algorithm': algo_name,
            'Accuracy': accuracy,
            'Confusion_Matrix': conf_matrix,
            'Precision_Positive': class_report['1']['precision'],
            'Recall_Positive': class_report['1']['recall'],
            'F1-Score_Positive': class_report['1']['f1-score'],
            'Precision_Negative': class_report['0']['precision'],
            'Recall_Negative': class_report['0']['recall'],
            'F1-Score_Negative': class_report['0']['f1-score']
        })

# Convert results to a DataFrame for better readability
results_df = pd.DataFrame(results)

# Display the results
print(results_df)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [04:16:07] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [04:16:27] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: Undefin

         Vectorizer            Algorithm  Accuracy         Confusion_Matrix  \
0   CountVectorizer  Logistic Regression  0.931202  [[164, 65], [34, 1176]]   
1   CountVectorizer          Naive Bayes  0.926338  [[155, 74], [32, 1178]]   
2   CountVectorizer        Random Forest  0.941626  [[170, 59], [25, 1185]]   
3   CountVectorizer                  SVM  0.922863  [[172, 57], [54, 1156]]   
4   CountVectorizer             AdaBoost  0.911049  [[141, 88], [40, 1170]]   
5   CountVectorizer              XGBoost  0.941626  [[171, 58], [26, 1184]]   
6            TF-IDF  Logistic Regression  0.928423  [[143, 86], [17, 1193]]   
7            TF-IDF          Naive Bayes  0.873523   [[48, 181], [1, 1209]]   
8            TF-IDF        Random Forest  0.947186  [[171, 58], [18, 1192]]   
9            TF-IDF                  SVM  0.936762  [[167, 62], [29, 1181]]   
10           TF-IDF             AdaBoost  0.915914  [[146, 83], [38, 1172]]   
11           TF-IDF              XGBoost  0.938151  

**Deep Learning Algorithm**

In [32]:
!pip install tensorflow

In [33]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GRU, Dense, Dropout, Embedding, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import confusion_matrix, classification_report

In [34]:
# Define a function to create the CNN + GRU model
def create_cnn_gru_model(input_length, vocab_size=5000):
    model = Sequential()

    # Embedding layer for word vectors
    model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=input_length))

    # CNN layer for feature extraction
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))

    # GRU layer for sequence processing
    model.add(GRU(128, return_sequences=True))
    model.add(Dropout(0.5))

    # Global max pooling for dimensionality reduction
    model.add(GlobalMaxPooling1D())

    # Fully connected output layer
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Sigmoid for binary classification

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [35]:
# Function to train and evaluate the CNN + GRU model with CountVectorizer
def train_evaluate_model_count(X_train, X_test, y_train, y_test):
    # Vectorizer
    vectorizer = CountVectorizer(max_features=5000)

    # Transform the data using the vectorizer
    X_train_vectorized = vectorizer.fit_transform(X_train).toarray()
    X_test_vectorized = vectorizer.transform(X_test).toarray()

    # Determine the max sequence length from the training data
    max_length = X_train_vectorized.shape[1]

    # Create and train the CNN + GRU model
    model = create_cnn_gru_model(max_length)

    model.fit(X_train_vectorized, y_train, epochs=5, batch_size=32, validation_split=0.1)

    # Predict on the test set
    y_pred = (model.predict(X_test_vectorized) > 0.5).astype("int32").flatten()

    # Store and print results
    print("Confusion Matrix for CountVectorizer:")
    print(confusion_matrix(y_test, y_pred))
    print("Classification Report for CountVectorizer:")
    print(classification_report(y_test, y_pred, target_names=['Not Recommended', 'Recommended']))

# Run for CountVectorizer
train_evaluate_model_count(X_train, X_test, y_train, y_test)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
162/162 ━━━━━━━━━━━━━━━━━━━━ 1090s 7s/step - accuracy: 0.8142 - loss: 0.4720 - val_accuracy: 0.8438 - val_loss: 0.4798
Epoch 2/5
162/162 ━━━━━━━━━━━━━━━━━━━━ 1084s 7s/step - accuracy: 0.8344 - loss: 0.4425 - val_accuracy: 0.8438 - val_loss: 0.4856
Epoch 3/5
162/162 ━━━━━━━━━━━━━━━━━━━━ 1073s 7s/step - accuracy: 0.8379 - loss: 0.4379 - val_accuracy: 0.8438 - val_loss: 0.4416
Epoch 4/5
162/162 ━━━━━━━━━━━━━━━━━━━━ 1093s 7s/step - accuracy: 0.8364 - loss: 0.4414 - val_accuracy: 0.8438 - val_loss: 0.4386
Epoch 5/5
162/162 ━━━━━━━━━━━━━━━━━━━━ 1039s 6s/step - accuracy: 0.8301 - loss: 0.4451 - val_accuracy: 0.8438 - val_loss: 0.4532
45/45 ━━━━━━━━━━━━━━━━━━━━ 55s 1s/step
Confusion Matrix for CountVectorizer:
[[   0  229]
 [   0 1210]]
Classification Report for CountVectorizer:
                 precision    recall  f1-score   support

Not Recommended       0.00      0.00      0.00       229
    Recommended       0.84      1.00      0.91      1210

       accuracy                    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [36]:

# Function to train and evaluate the CNN + GRU model with TF-IDF
def train_evaluate_model_tfidf(X_train, X_test, y_train, y_test):
    # Vectorizer
    vectorizer = TfidfVectorizer(max_features=5000)

    # Transform the data using the vectorizer
    X_train_vectorized = vectorizer.fit_transform(X_train).toarray()
    X_test_vectorized = vectorizer.transform(X_test).toarray()

    # Determine the max sequence length from the training data
    max_length = X_train_vectorized.shape[1]

    # Create and train the CNN + GRU model
    model = create_cnn_gru_model(max_length)

    model.fit(X_train_vectorized, y_train, epochs=5, batch_size=32, validation_split=0.1)

    # Predict on the test set
    y_pred = (model.predict(X_test_vectorized) > 0.5).astype("int32").flatten()

    # Store and print results
    print("Confusion Matrix for TF-IDF:")
    print(confusion_matrix(y_test, y_pred))
    print("Classification Report for TF-IDF:")
    print(classification_report(y_test, y_pred, target_names=['Not Recommended', 'Recommended']))

# Run for TF-IDF
train_evaluate_model_tfidf(X_train, X_test, y_train, y_test)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
162/162 ━━━━━━━━━━━━━━━━━━━━ 1052s 6s/step - accuracy: 0.8330 - loss: 0.4898 - val_accuracy: 0.8438 - val_loss: 0.4677
Epoch 2/5
162/162 ━━━━━━━━━━━━━━━━━━━━ 1089s 6s/step - accuracy: 0.8349 - loss: 0.4534 - val_accuracy: 0.8438 - val_loss: 0.4787
Epoch 3/5
162/162 ━━━━━━━━━━━━━━━━━━━━ 1035s 6s/step - accuracy: 0.8379 - loss: 0.4446 - val_accuracy: 0.8438 - val_loss: 0.4741
Epoch 4/5
162/162 ━━━━━━━━━━━━━━━━━━━━ 1055s 6s/step - accuracy: 0.8357 - loss: 0.4489 - val_accuracy: 0.8438 - val_loss: 0.4896
Epoch 5/5
162/162 ━━━━━━━━━━━━━━━━━━━━ 1100s 6s/step - accuracy: 0.8344 - loss: 0.4513 - val_accuracy: 0.8438 - val_loss: 0.4828
45/45 ━━━━━━━━━━━━━━━━━━━━ 53s 1s/step
Confusion Matrix for TF-IDF:
[[   0  229]
 [   0 1210]]
Classification Report for TF-IDF:
                 precision    recall  f1-score   support

Not Recommended       0.00      0.00      0.00       229
    Recommended       0.84      1.00      0.91      1210

       accuracy                           0.84      1

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
